# Assigment 7

This assigment will be graded if everything works well. I will run the script as once and everything should be done without errors and mistakes. I should be able to run your scripts in my computer and get all the results. **USE RELATIVE PATHS**. An error or exception or anything that breaks the code will means NO GRADE (0). Additionally, you are not able to modify any file handly. It also means NO GRADE (0). Comment everything you think will help others read your script. We expect 0 errors using GitHub. Everything will be graded!

**ASK EVERYTHING! WE ARE HERE TO HELP YOU!**

**GET YOUR GOOGLE API AND TOKEN. YOU WILL NEED THEM TO DO THIS TASK.**


1. Import Data from [this url](https://www.bbva.pe/content/dam/public-web/peru/documents/personas/canales-de-atencion/oficinas/Oficinas-BBVA-abiertas-23.10.20.pdf). This dataset is in PDF format. You have to convert to PandasDataFrame and keep only the BBVA offices in LIMA.
2. Use GoogleMaps API and geocode all the BBVA offices. For those offices that Google API gets no information, use internet and get the latitude and longitude handly and add them to dataset.
3. Use Google API to find the driving time (best guess) from all the group members' address and all the LIMA BBVA offices.
4. Finally, you have to give a report which offices are the most closest and furthest to every group member's address.

# Importing necessary packages

In [ ]:
# For sending GET requests from the API
import requests

# For saving access tokens and for file management when creating and adding to the dataset
import os

# For dealing with json responses we receive from the API
import json

# For displaying the data after
import pandas as pd
import numpy as np

# For saving the response data in CSV format
import csv

# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata

#To add wait time between requests
import time
import requests

#maps
import googlemaps

# Other packages
import urllib.request
from tqdm import tqdm_notebook as tqdm
import re

# I) Generating the DF with BBVA offices

In [ ]:
# Getting the file "bbva_list.xlsx" from the "_data" folder 
file_path = r"../../_data/bbva_list.xlsx" 

# Creating Data Frame of the BBVA offices's list
data_excel = pd.read_excel(file_path)
data_lima = pd.DataFrame(data_excel)